In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import logging
import unittest
from binascii import hexlify
from doctest import testmod
from os import environ
from sys import modules, stderr, stdout
from time import sleep
from pyftdi import FtdiLogger
from pyftdi.spi import SpiController, SpiIOError
import time


In [5]:
# Instanciate a SPI controller for ice40

#https://eblot.github.io/pyftdi/pinout.html
    
spi = SpiController()

# Configure the first interface (IF/1) of the FTDI device as a SPI master
spi.configure('ftdi://ftdi:2232h/2')
#spi.configure('ftdi://::/1')

# Get a port to a SPI slave w/ /CS on A*BUS3 and SPI mode 0 @ 12MHz
slave = spi.get_port(cs=0, freq=12E6, mode=0)

# RESET the ICE40 -> BDBUS7 // # CS4/GPIO7

gpio = spi.get_gpio()
DIR_PREVIOUS  = gpio.direction
print( "Dir:" + bin( DIR_PREVIOUS ) )
spi._spi_mask = 0b0001111
#print(bin(spi._spi_mask))
gpio.set_direction(0b10000000, 0b10000000)
print( "Dir:" + bin( gpio.direction ) )
print("====")
print("GPIO STATUS: ",gpio.read())
gpio.write(0b10000000)
print("GPIO STATUS: ",gpio.read())
gpio.write(0b00000000)
print("GPIO STATUS: ",gpio.read() & 0b10000000)
time.sleep(1)
gpio.write(0b10000000)
print("GPIO STATUS: ",gpio.read() & 0b10000000)
time.sleep(1)

Dir:0b1111011
Dir:0b11111011
====
GPIO STATUS:  128
GPIO STATUS:  128
GPIO STATUS:  0
GPIO STATUS:  128


In [4]:
# Instanciate a SPI controller
spi2 = SpiController()

# Configure the first interface (IF/1) of the FTDI device as a SPI master
spi2.configure('ftdi://ftdi:2232h/1')

# Get a port to a SPI slave w/ /CS on A*BUS3 and SPI mode 0 @ 12MHz
slave = spi2.get_port(cs=1, freq=12E6, mode=0)

# RESET the ICE40 -> BDBUS7
# Request the JEDEC ID from the SPI slave
jedec_id = slave.exchange([0x9f], 3).tobytes()
print(jedec_id)

b'\xff\xff\xff'
